# Triage & Vuln Analysis

This notebook demonstrates the security-focused fields added in v1.1.0: severity, found-in versions, in-production flags, view counts, and issue relationships.

In [10]:
from buganize import Buganize

## Tracker Selection

By default, queries search all public trackers. Use `tracker` to scope to a specific one.

In [ ]:
# Search only Chromium issues
async with Buganize(tracker="chromium") as client:
    result = await client.search("type:vulnerability status:open", page_size=25)

print(f"Chromium open vulns: {result.total_count}")
for issue in result.issues[:5]:
    print(f"#{issue.id} [{issue.status.name}] ({issue.priority.name}) {issue.title}")

## Severity vs Priority

Severity and priority are independent fields. Security issues often have severity diverge from priority.

In [ ]:
async with Buganize(tracker="chromium") as client:
    result = await client.search("type:vulnerability", page_size=50)

for issue in result.issues:
    sev = issue.severity.name if issue.severity is not None else "-"
    diverged = (
        " ***"
        if issue.severity is not None and issue.severity.value != issue.priority.value
        else ""
    )
    print(
        f"#{issue.id}  P={issue.priority.name}  S={sev}  {issue.title[:60]}{diverged}"
    )

## Triage Fields

Fields useful for triaging: which version the bug was found in, whether it's been seen in production, and who's collaborating.

In [ ]:
async with Buganize(tracker="chromium") as client:
    issue = await client.issue(40060244)

print(f"#{issue.id} — {issue.title}")
print(f"  Found in:      {issue.found_in or 'not set'}")
print(f"  In production: {issue.in_prod}")
print(f"  Collaborators: {issue.collaborators or 'none'}")

## Issue Relationships

See which issues are blocked by or duplicated from the current one.

In [ ]:
print(f"#{issue.id} — {issue.title}")
print(f"  Blocking:   {issue.blocking_issue_ids or 'none'}")
print(f"  Duplicates: {issue.duplicate_issue_ids or 'none'}")

## Activity Signals

View counts help gauge how much attention an issue is getting. Useful for spotting trending bugs.

In [ ]:
async with Buganize(tracker="chromium") as client:
    result = await client.search("type:vulnerability status:open", page_size=25)

# Sort by 7-day views descending — hottest issues first
hot = sorted(result.issues, key=lambda i: i.views_7d, reverse=True)

print("Most-viewed open vulns (last 7 days):")
for issue in hot[:10]:
    print(f"  {issue.views_7d:>4} views  #{issue.id}  {issue.title[:50]}")